# 0.9_ss_ぐらふ

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_rows = 5

In [3]:
# from, to, cost
distances = [
    ("a", "b", 10),
    ("a", "f", 2),
    ("b", "c", 5),
    ("c", "f", 7),
    ("c", "d", 8),
    ("c", "e", 1),
    ("d", "e", 5),
    ("e", "f", 3),
    ("f", "g", 1),
]
# 双方向
double_distances = distances + [(end, start, cost) for start, end, cost in distances]

In [4]:
E = len(distances)
E

9

In [5]:
df_dist = pd.DataFrame(double_distances, columns=['start', 'end', 'cost'])
df_dist

,start,end,cost
0,a,b,10
1,a,f,2
...,...,...,...
16,f,e,3
17,g,f,1


In [6]:
nodes = list(set(df_dist.loc[:, ['start', 'end']].values.flatten()))
nodes.sort()
V = len(nodes)
V

7

In [7]:
node_map = {item: nodes.index(item) for item in nodes}

In [8]:
df_dist['from'] = df_dist.start.map(node_map)
df_dist['to'] = df_dist.end.map(node_map)

In [9]:
df_dist

,start,end,cost,from,to
0,a,b,10,0,1
1,a,f,2,0,5
...,...,...,...,...,...
16,f,e,3,5,4
17,g,f,1,6,5


## Make cost matrix

In [10]:
cost = np.ones((V, V)) * np.inf

for index, row in df_dist.iterrows():
    cost[row['from']][row['to']] = row['cost']

In [11]:
cost

array([[inf, 10., inf, inf, inf,  2., inf],
       [10., inf,  5., inf, inf, inf, inf],
       [inf,  5., inf,  8.,  1.,  7., inf],
       [inf, inf,  8., inf,  5., inf, inf],
       [inf, inf,  1.,  5., inf,  3., inf],
       [ 2., inf,  7., inf,  3., inf,  1.],
       [inf, inf, inf, inf, inf,  1., inf]])

In [12]:
# 集合Xから辺への最小コスト
mincost = np.ones(V) * np.inf
mincost[0] = 0

# 使われているか
used = np.zeros(V, dtype=bool)

# 結果
res = 0

while(True):
    v = -1
    
    # Xに属さない頂点のうちから、辺のコストが最小になる頂点を探す
    for u in range(V):
        # if !used[u] and ((v == -1) or (mincost[u] < mincost[v])):
        #    v = u
        # 使われているならスキップ
        if used[u]:
            continue
        
        # 初期値なら u を入れる
        if v == -1:
            v = u
        
        # コストが最小なら、u を入れる
        if mincost[u] < mincost[v]:
            v = u
    
    # 更新されなかった場合
    if v == -1:
        break
    
    # v を集合Xに含む
    used[v] = True
    res += mincost[v]
    
    # vが含まれたので更新
    for u in range(V):
        mincost[u] = min(mincost[u], cost[v][u])

print(res)

17.0


In [13]:
mincost

array([0., 5., 1., 5., 1., 1., 1.])

In [14]:
distances

[('a', 'b', 10),
 ('a', 'f', 2),
 ('b', 'c', 5),
 ('c', 'f', 7),
 ('c', 'd', 8),
 ('c', 'e', 1),
 ('d', 'e', 5),
 ('e', 'f', 3),
 ('f', 'g', 1)]

# Use union find tree

In [15]:
double_distances.sort(key=lambda x: x[2])

In [16]:
double_distances

[('c', 'e', 1),
 ('f', 'g', 1),
 ('e', 'c', 1),
 ('g', 'f', 1),
 ('a', 'f', 2),
 ('f', 'a', 2),
 ('e', 'f', 3),
 ('f', 'e', 3),
 ('b', 'c', 5),
 ('d', 'e', 5),
 ('c', 'b', 5),
 ('e', 'd', 5),
 ('c', 'f', 7),
 ('f', 'c', 7),
 ('c', 'd', 8),
 ('d', 'c', 8),
 ('a', 'b', 10),
 ('b', 'a', 10)]

In [17]:
from src.union_find_tree import UnionFindTree

In [51]:
tree = UnionFindTree(V)
res = 0
distances.sort(key=lambda x: x[2])

for u_, v_, cost in distances:
    u = node_map[u_]
    v = node_map[v_]
    if not tree.same(u, v):
        tree.unite(u, v)
        res += cost

In [54]:
tree.par

array([2, 2, 2, 2, 2, 2, 5])

# Second best

In [55]:
distances = [
    (0, 1, 100),
    (1, 2, 250),
    (1, 3, 200),
    (2, 3, 100),
]

In [60]:
G = [
    [(1, 100), ],
    [(0, 100), (2, 250), (3, 200),],
    [(1, 250), (3, 100), ],
    [(1, 200), (2, 100), ],
]

In [59]:
import heapq

In [95]:
N = 4
R = 4
dist = np.ones(N) * np.inf
dist[0] = 0
dist2 = np.ones(N) * np.inf

# que is dist, vector
que = []
heapq.heappush(que, (0, 0))

while(que):
    d, v = heapq.heappop(que)
    print('v', v)
    
    # 3番手以降の解
    if dist2[v] < d:
        print('skip', dist2[v], d)
        continue
        
    for to, cost in G[v]:
        print('v', v, 'to', to)
        d2 = d + cost
        # 最小値の更新
        if dist[to] > d2:
            print('bef', que, v, '->', to, d2, dist, dist2)
            dist[to], d2 = d2, dist[to]
            heapq.heappush(que, (dist[to], to))
            print('min', que, v, '->', to, d2, dist, dist2)
        
        # 2番目のあたいの更新
        if dist2[to] > d2 > dist[to]:
            print('bef', que, v,'->', to, d2, dist, dist2)
            dist2[to] = d2
            heapq.heappush(que, (dist2[to], to))
            print('2min', que, v,'->', to, d2, dist, dist2)

v 0
v 0 to 1
bef [] 0 -> 1 100 [ 0. inf inf inf] [inf inf inf inf]
min [(100.0, 1)] 0 -> 1 inf [  0. 100.  inf  inf] [inf inf inf inf]
v 1
v 1 to 0
bef [] 1 -> 0 200.0 [  0. 100.  inf  inf] [inf inf inf inf]
2min [(200.0, 0)] 1 -> 0 200.0 [  0. 100.  inf  inf] [200.  inf  inf  inf]
v 1 to 2
bef [(200.0, 0)] 1 -> 2 350.0 [  0. 100.  inf  inf] [200.  inf  inf  inf]
min [(200.0, 0), (350.0, 2)] 1 -> 2 inf [  0. 100. 350.  inf] [200.  inf  inf  inf]
v 1 to 3
bef [(200.0, 0), (350.0, 2)] 1 -> 3 300.0 [  0. 100. 350.  inf] [200.  inf  inf  inf]
min [(200.0, 0), (350.0, 2), (300.0, 3)] 1 -> 3 inf [  0. 100. 350. 300.] [200.  inf  inf  inf]
v 0
v 0 to 1
bef [(300.0, 3), (350.0, 2)] 0 -> 1 300.0 [  0. 100. 350. 300.] [200.  inf  inf  inf]
2min [(300.0, 1), (350.0, 2), (300.0, 3)] 0 -> 1 300.0 [  0. 100. 350. 300.] [200. 300.  inf  inf]
v 1
v 1 to 0
v 1 to 2
bef [(300.0, 3), (350.0, 2)] 1 -> 2 550.0 [  0. 100. 350. 300.] [200. 300.  inf  inf]
2min [(300.0, 3), (350.0, 2), (550.0, 2)] 1 -> 2 550.

In [93]:
dist2

array([200., 300., 400., 450.])

In [94]:
dist

array([  0., 100., 350., 300.])

In [81]:
3 > 4 > 5

False

In [82]:
5 > 4 > 3

True

In [83]:
5 < 3 > 2

False